# 9. 네이버톡톡 챗봇 만들기

## 9.1 네이버톡톡 챗봇

1. 네이버톡톡 chatbot api는 `HTTPS만 지원`한다.
2.  따라서 소유중인 도메인이 SSL을 적용할 수 없다면 네이버톡톡 챗봇을 사용할 수 없다.

### 9.1.1 네이버톡톡 가입

1. `partner.talk.naver.com`

<img src="./images/1101.네이버톡톡 가입하기_01.png" /> 
<img src="./images/1101.네이버톡톡 가입하기_02_계정만들기.PNG" /> 
<img src="./images/1101.네이버톡톡 가입하기_03_프로필작성.PNG" /> 
<img src="./images/1101.네이버톡톡 가입하기_04_톡톡시작하기.PNG" /> 
<img src="./images/1101.네이버톡톡 가입하기_05_네이버톡톡계정목록.PNG" /> 


### 9.1.2 네이버톡톡 chatbot API 설정

1. chatbot API설정에 필요한 이벤트 받은 URL과 챗봇에서 사용할 이벤트를 선택
   - 이벤트받을 URL : https://"chatbot api server domain"/query/NAVER
   - 이벤트선택 : (button)이벤트변경 > (check)`[send, open, leave, friend]` 선택
   - API인증키 생성
     - API사용에 필요한 업체정보를 요구 받을 수 있다.
     - 개인일 경우 `본인의 이름 or 블로그주소를 입력`하면 chatbot API를 사용할 수 있다.

2. http://guide.thecloudgate.io/wp-content/uploads/2021/07/navertalktalk-11.png
3. 참고사이트 : http://guide.thecloudgate.io/docs/guide/channel-integration/navertalktalk/   

## 9.2 네이버톡톡 Chat Bot API

* ChatBot API 동작구조 : https://github.com/navertalk/chatbot-api
<img src="./images/1102.네이버톡톡 01_chatbotapi_structure.PNG" /> 

### 9.2.1 이벤트명세

1. 네이버톡톡에서 `이벤트는 Webhook을 통해 chatbot api server에 전달`된다.
   - 카카오봇시스템이 사용자 발화 데이터를 스킬 API로 전잔하는 것과 비슷한 개념
2. `이벤트요청메시지는 HTTPS POST요청으로 전달`되며 body에 JSON포맷으로 저장되어 있다.
3. 이벤트요청메시지 기본구조
```json
{
    "event": "<이벤트명>",
    "options": { /* 추가속성 */ },
    "user": "<사용자식별값>"
}
```
* event : 선택한 `이벤트종류`가 들어 온다.
* options: `특정 이벤트에서만 사용되는 추가적인 속성`이 포함
* user : 네이버톡톡 사용자를 식별할 수 있는 `유니크키값`이 포함

4. 이벤트요청받은 후 `챗봇엔진에서 처리된 결과를 응답메시지로 전송`해야 한다.
   - 카카오오픈빌더의 스킬서버에서 응답스킬을 전달하는 것과 동일한 개념
6. 이벤트응답메시지 기본구조
```json
{
    "event": "send",
    "textContent": "Hello World!!"
}
```
* 네이버톡톡은 사용자메시지(발화)에 응답하는 형태에선 `동기식과 비동기식`이 있다.
  - 동기식응답   : `사용자 이벤트 요청에 바로 응답 메시지를 주는 방법`이며 `응답이 1개이거나 5초 이내에 결과메시지`를 줄 수 있을 때 사용한다.
  - 비동기식응답 : `사용자 이벤트 요청에 HTTP상태코드(200)응답후 보내기 API를 통해 응답메시지를 전달하는 방법`이며 `여러건의 메시지나 일정시간이후에 메시지전송`시 사용
  - 네이버톡톡에서 지원하는 이벤트 종류
  |이벤트|설명|
  |:------:|:---------------------------------------|
  |open|사용자가 채팅창에 진입했을 때 전달되는 이벤트. open이벤트의 options필드에 사용사 유집정보가 포함|
  |leave|사용자가 채팅창에서 나가기를 누르면 발생하는 이벤트. 사용자가 채팅방을 나가는 시점을 확인할 수 있다.|
  |friend|사용자가 톡톡 계정에서 친구추가 또는 친구철회를 누르면 발생하는 이벤트. friend이벤트의 options필으데 친구추가 및 친구철회정보가 포함|
  |send|send이벤트는 사용자와 챗봇이 메시지를 전송할 때 발생하는 이벤트. send이벤트의 메시지타입에 따라 챗봇에 표현되는 말풍선 모양이 달라진다.|

### 9.2.2 메시지 타입 명세

* 챗봇API서버에서 톡톡챗봇플랫폼으로 `응답메시지를 전달할 때는 send 이벤트를 사용`한다.
* 이때, 사용자에게 보여줄 챗봇메시지의 종류를 선택할 수 있댜ㅏ.

1. `textContent`는 챗봇이 사용자에게 텍스트메시지를 전송할 때나 사용자가 챗봇에 텍스트메시지를 전송할 때 사용하는 메시지 타입이다.

* 사용자 -> 챗봇 send 이벤트 : textContent메시지 타입 예시
    ```json
    {
        "event": "send",
        "user": "<사용자 식별값>",
        "textContent": {
            "text": "안녕",
            "inputType": "typing"
        }
    }
    ```
    - text : 사용자가 입력한 메시지(발화)가 저장
    - inputType : 사용자가 어떤 방식으로 메시지를 입력했는지 표현
      - typing : 사용자가 직접 채팅창에 메시지를 입력해 보낸 경우

* 챗봇 send 이벤트 -> 사용자 : textContent메시지 타입 예시
    ```json
    {
        "event": "send",
        "user": "<사용자 식별값>",
        "textContent": {
            "text": "네, 저도 반가워요!!",
        }
    }
    ```
2. `imageContent`는 이미지를 전송할 수 있는 메시지 타입이다.
   - 이미지링크주소를 포함하고 고정된 이미지 형태로 전달된다.

* 사용자 -> 챗봇 send 이벤트 : imageContent 타입 예시
    ```json
    {
        "event": "send",
        "user": "<사용자 식별값>",
        "imageContent": {
            "imageUrl": "<네이버 서버에 저장된 이미지 url>",
            "width": "<이미지 너비>",
            "height": "<이미지 높이>",
        }
    }
    ```
    - text : 사용자가 입력한 메시지(발화)가 저장
    - inputType : 사용자가 어떤 방식으로 메시지를 입력했는지 표현
      - typing : 사용자가 직접 채팅창에 메시지를 입력해 보낸 경우

* 챗봇 send 이벤트 -> 사용자 : imageContent메시지 타입 예시
    ```json
    {
        "event": "send",
        "user": "<사용자 식별값>",
        "imageContent": {
            "imageUrl": "<사용자에게 전송할 이미지 url>",
        }
    }
    ```

### 9.2.3 보내기 API작성

* 카카오톡 챗봇은 동기식방식이다.
* `네이버톡톡은 메시지를 보낼 때 사용자에게 비동기식으로 응답메시지를 전달할 수 있는 API를 추가로 제공`한다.
  - 메시지를 보낼 대상의 사용자식별값만 알 고 있다면 자유롭게 메시지를 전송할 수 있다.
  - 주로 챗봇이 사용자에게 먼저 대화를 요청하거나 알림을 주고 싶을 때 사용
  - 사용자 동의만 있다면 광고 메시지도 보낼 수 있다.
  - 동기식처럼 사용자 이벤트에 대한 응답으로도 사용할 수 있다.
  - 동기식에 비해 편리한 점이 많아 `비동기식 응답방식으로 실습을 진행!!`

#### 보내기 API 예제

```python
import requests, json

# 1. 보내기 API 인증키
authorization_key ='<보내기 API 인증키>'
headers = {
    'Content-Type': 'application/json;charset=UTF-8',
    'Authorization': self.authorization_key,
}

# 2. 사용자 식별값, 보낼 메시지 정의
user_key = '<메시지 전송대상 사용자 식별값>'
data = {
    "event": "send",
    "user": user_key,
    "textContent": {
        "text": "Hello World!! ^^;"
    }    
}

# 3. 보내기 API 호출
message = json.dumps(data)  # JSON 문자열 변경
response =  requests.post(
    'https://gw.talk.naver.com/chatbot/v1/event',
    headers=headers,
    data=message)
print(response.status_code)  # 성공시 200
print(response.text)         # 성공시 Hello World!! ^^; 반환
```

* 오류명세 : https://github.com/navertalk/chatbot-api?tab=readme-ov-file#오류-명세

## 9.3 챗봇 API서버에 네이버톡톡 연동

### 9.3.1 챗봇 API 서버 수정

##### A. 05.python\chatbot_api\NaverEvent.py

```python
import requests, json

class NaverEvent:
    def __init__(self, authorization):
        self.authorization_key = authorization

    # 텍스트 컨텐트 출력 요소
    def textContentComponent(self, text):
        return {
            "textContent": {
                "text": text
            }
        }

    # 이미지 컨텐트 출력 요소
    def imageContentComponent(self, imageUrl):
        return {
            "imageContent": {
                "imageUrl": imageUrl
            }
        }

    # 보내기 API로 메시지 전송
    def send_message(self, user_key, component):
        headers = {
            'Content-Type': 'application/json;charset=UTF-8',
            'Authorization': self.authorization_key,
        }

        data = {
            "event": "send",
            "user": user_key,
        }
        data.update(component)

        # 보내기 API 호출
        message = json.dumps(data)  # JSON 문자열 변경
        return requests.post(
            'https://gw.talk.naver.com/chatbot/v1/event',
            headers=headers,
            data=message)

    # 사용자에게 응답 전송
    def send_response(self, dst_user_key, bot_resp):
        # 이미지 답변이 텍스트 답변보다 먼저 출력 됨
        # 이미지 답변이 있는 경우
        if bot_resp['AnswerImageUrl'] is not None:
            image = self.imageContentComponent(bot_resp['AnswerImageUrl'])
            self.send_message(user_key=dst_user_key, component=image)

        # 텍스트 답변이 있는 경우
        if bot_resp['Answer'] is not None:
            text = self.textContentComponent(bot_resp['Answer'])
            self.send_message(user_key=dst_user_key, component=text)

        return json.dumps({}), 200
```

##### B. 05.python\chatbot_api\app.py

```python
from flask import Flask, request, jsonify, abort
import socket|
import json

# 챗봇 엔진 서버 접속 정보
host = "127.0.0.1"  # 챗봇 엔진 서버 IP 주소
port = 5050  # 챗봇 엔진 서버 통신 포트

# Flask 어플리케이션
app = Flask(__name__)


# 챗봇 엔진 서버와 통신
def get_answer_from_engine(bottype, query):
    # 챗봇 엔진 서버 연결
    mySocket = socket.socket()
    mySocket.connect((host, port))

    # 챗봇 엔진 질의 요청
    json_data = {
        'Query': query,
        'BotType': bottype
    }
    message = json.dumps(json_data)
    mySocket.send(message.encode())

    # 챗봇 엔진 답변 출력
    data = mySocket.recv(2048).decode()
    ret_data = json.loads(data)

    # 챗봇 엔진 서버 연결 소켓 닫기
    mySocket.close()

    return ret_data


@app.route('/', methods=['GET'])
def index():
    return 'hello', 200


# 챗봇 엔진 query 전송 API
@app.route('/query/<bot_type>', methods=['POST'])
def query(bot_type):
    body = request.get_json()

    try:
        if bot_type == 'TEST':
            # 챗봇 API 테스트
            ret = get_answer_from_engine(bottype=bot_type, query=body['query'])
            return jsonify(ret)

        elif bot_type == "KAKAO":
            # 카카오톡 스킬 처리
            body = request.get_json()
            utterance = body['userRequest']['utterance']
            ret = get_answer_from_engine(bottype=bot_type, query=utterance)

            from KakaoTemplate import KakaoTemplate
            skillTemplate = KakaoTemplate()
            return skillTemplate.send_response(ret)

        elif bot_type == "NAVER":
            # 네이버톡톡 이벤트 처리
            body = request.get_json()
            user_key = body['user']
            event = body['event']

            from NaverEvent import NaverEvent
            authorization_key = '<보내기 API 인증키>'
            naverEvent = NaverEvent(authorization_key)

            if event == "open":
                # 사용자가 채팅방 들어왔을 때 처리
                print("채팅방에 유저가 들어왔습니다.")
                return json.dumps({}), 200

            elif event == "leave":
                # 사용자가 채팅방 나갔을 때 처리
                print("채팅방에 유저가 나갔습니다.")
                return json.dumps({}), 200

            elif event == "send":
                # 사용자가 챗봇에게 send 이벤트를 전송했을 때
                user_text = body['textContent']['text']
                ret = get_answer_from_engine(bottype=bot_type, query=user_text)
                return naverEvent.send_response(user_key, ret)

            return json.dumps({}), 200

        else:
            # 정의되지 않은 bot type인 경우 404 오류
            abort(404)

    except Exception as ex:
        # 오류 발생시 500 오류
        abort(500)


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)
```